In [1]:
import cv2
import tensorflow as tf
import numpy as np
import os
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

In [2]:
def predict_face(reshaped_face, model):
    CATEGORIES = ['real', 'fake']
    prediction = model.predict(reshaped_face)
    return CATEGORIES[int(prediction[0][0])]

In [3]:
def reshape_face(face):
    IMG_SIZE = 100
    img_array = cv2.imread(face) 
    new_array = cv2.resize(img_array, (IMG_SIZE, IMG_SIZE))
    return new_array.reshape(-1, IMG_SIZE, IMG_SIZE, 3)

In [4]:
def get_face(image_path):
    file_name = 'haarcascade_frontalface_alt.xml'
    lib_dir_path = r"C:\Users\adria\Desktop\hackathon\aplikacja\venv\Lib\site-packages\cv2\data"
    face_cascade_path  = os.path.join(lib_dir_path, file_name)
    face_cascade = cv2.CascadeClassifier(face_cascade_path)
    image = cv2.imread(image_path)
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    
    faces = face_cascade.detectMultiScale(gray, 1.3, 5)
    face_exist = False
    for (x,y,w,h) in faces:
        face_exist = True
        image = image[y:y+h, x:x+w]
    if face_exist:
        return image
    else:
        return None

In [5]:
def predict_image(source_video_path, model):
    face = get_face(source_video_path)
    if face is not None:
        cv2.imwrite('temporary.png', face)
        reshaped_face = reshape_face('temporary.png')
        result = predict_face(reshaped_face, model)
        print(result)
        return result

In [6]:
model = tf.keras.models.load_model('mesonet.model')

In [7]:
source_video_path = r"C:\Users\adria\Desktop\hackathon\fake1.jpg"
predict_image(source_video_path, model)

fake


'fake'

In [8]:
source_video_path = r"C:\Users\adria\Desktop\hackathon\real1.jpg"
predict_image(source_video_path, model)

real


'real'

In [9]:
source_video_path = r"C:\Users\adria\Desktop\hackathon\fake2.jpg"
predict_image(source_video_path, model)

fake


'fake'

In [10]:
source_video_path = r"C:\Users\adria\Desktop\hackathon\real2.jpg"
predict_image(source_video_path, model)

real


'real'

In [16]:
import requests
from PIL import Image
from io import BytesIO

In [137]:
def fake_rectangle(image_path):
    file_name = 'haarcascade_frontalface_alt.xml'
    lib_dir_path = r"C:\Users\adria\Desktop\hackathon\aplikacja\venv\Lib\site-packages\cv2\data"
    face_cascade_path  = os.path.join(lib_dir_path, file_name)
    face_cascade = cv2.CascadeClassifier(face_cascade_path)
    image = cv2.imread(image_path)
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    
    faces = face_cascade.detectMultiScale(gray, 1.3, 5)
    face_exist = False
    for (x,y,w,h) in faces:
        face_exist = True
        
        # font
        font = cv2.FONT_HERSHEY_SIMPLEX

        # org
        org = (x,y+h)

        # fontScale
        fontScale = 3

        # Blue color in BGR
        color = (0, 0, 255)

        # Line thickness of 2 px
        thickness = 3

        # Using cv2.putText() method
        image = cv2.putText(image, 'FAKE', org, font, 
                       fontScale, color, thickness, cv2.LINE_AA)
    
        cv2.rectangle(image,(x,y),(x+w,y+h),(0,0,255),3)
    if face_exist:
        return image
    else:
        return None

In [153]:
next_photo = requests.get("https://bhl22-delphic-backend.azurewebsites.net/api/picture/next")
print(next_photo.status_code)
if next_photo.status_code == 200:
    print(next_photo.json())
    pictureFileId = next_photo.json()['pictureFileId']
    picture = requests.get("https://bhl22-delphic-backend.azurewebsites.net/api/file/{}".format(pictureFileId))
    print(picture.status_code)
    if picture.status_code == 200:
        image = Image.open(BytesIO(picture.content))
        image.save("temopraryPIL.jpg")
        result = predict_image("temopraryPIL.jpg", model)
        if result == "fake":
            print(result)
            fakeImg = fake_rectangle("temopraryPIL.jpg")
            cv2.imwrite('fakeImg.png', fakeImg)
            pictureId = next_photo.json()['id']
            data_dict = {'file': fakeImg}
            files = {'file': open('fakeImg.png', 'rb')}
            send_request = requests.post("https://bhl22-delphic-backend.azurewebsites.net/api/picture/setAsFake/{}".format(pictureId),
                                        files=files)
            print(send_request.status_code)
            print(send_request.content)
        else:
            pictureId = next_photo.json()['id']
            data_dict = {'file': fakeImg}
            print(pictureId)
            send_request = requests.post("https://bhl22-delphic-backend.azurewebsites.net/api/picture/setAsReal/{}".format(pictureId))
            print(send_request.status_code)
            print(send_request.content)

200
{'id': '625150ad7b55d22a7879b85a', 'pictureFileId': '625150ad7b55d22a7879b859', 'createDate': '2022-04-09T09:23:57.216Z', 'verified': False, 'verifiedDate': None, 'fake': False, 'pictureIDs': []}
200
fake
fake
200
b'{"id":"625150ad7b55d22a7879b85a","pictureFileId":"625150ad7b55d22a7879b859","createDate":"2022-04-09T09:23:57.216Z","verified":true,"verifiedDate":"2022-04-09T09:25:23.561319828Z","fake":true,"pictureIDs":["625151037b55d22a7879b85f"]}'


In [144]:
sent_photo = requests.get("https://bhl22-delphic-backend.azurewebsites.net/api/picture/{}".format("62514a927b55d22a7879b843"))
print(next_photo.status_code)
if sent_photo.status_code == 200:
    print(sent_photo.json())
    print(sent_photo.content)
    image = Image.open(BytesIO(sent_photo.content))
    image.save("fakeeee.jpg")

200
{'id': '62514a927b55d22a7879b843', 'pictureFileId': '62514a927b55d22a7879b842', 'createDate': '2022-04-09T08:57:54.089Z', 'verified': True, 'verifiedDate': '2022-04-09T09:01:08.792Z', 'fake': False, 'pictureIDs': []}
b'{"id":"62514a927b55d22a7879b843","pictureFileId":"62514a927b55d22a7879b842","createDate":"2022-04-09T08:57:54.089Z","verified":true,"verifiedDate":"2022-04-09T09:01:08.792Z","fake":false,"pictureIDs":[]}'


UnidentifiedImageError: cannot identify image file <_io.BytesIO object at 0x0000026B666C00E0>